In [2]:
from itertools import chain
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [1]:
!pip freeze

absl-py==1.4.0
appnope==0.1.3
asttokens==2.0.8
astunparse==1.6.3
attrs==22.1.0
backcall==0.2.0
black==22.10.0
blis==0.7.8
bokeh==3.0.3
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.9.24
charset-normalizer==2.1.1
click==8.1.3
compress-json==1.0.8
confection==0.0.3
contourpy==1.0.7
cycler==0.11.0
cymem==2.0.6
debugpy==1.6.3
decorator==5.1.1
eli5==0.13.0
entrypoints==0.4
executing==1.1.1
flatbuffers==23.1.21
fonttools==4.38.0
fst-pso==1.8.1
FuzzyTM==2.0.5
gast==0.4.0
gensim==4.3.1
google-auth==2.16.1
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
graphviz==0.20.1
grpcio==1.51.3
h5py==3.8.0
hmmlearn==0.2.7
idna==3.4
importlib-metadata==6.0.0
importlib-resources==5.12.0
ipykernel==6.16.0
ipython==8.5.0
jedi==0.18.1
Jinja2==3.1.2
joblib==1.2.0
jupyter-core==4.11.1
jupyter_client==7.3.5
keras==2.11.0
kiwisolver==1.4.4
langcodes==3.3.0
libclang==15.0.6.1
livelossplot==0.5.5
Markdown==3.4.1
MarkupSafe==2.1.1
matplotlib==3.7.0
matplotlib-inline==0.1.6
miniful==0.0.6
murmurhash==1.0.8
mypy-ex

Inspeciona a classe com mais detalhe

In [293]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-LOC      0.787     0.074     0.135      1502
       I-LOC      0.000     0.000     0.000         0
      B-MISC      0.000     0.000     0.000         0
      I-MISC      0.000     0.000     0.000         0
       B-ORG      0.000     0.000     0.000         0
       I-ORG      0.000     0.000     0.000         0
       B-PER      0.381     0.005     0.010      1571
       I-PER      0.000     0.000     0.000         0

   micro avg      0.696     0.039     0.073      3073
   macro avg      0.146     0.010     0.018      3073
weighted avg      0.580     0.039     0.071      3073



## Hyperparameter Optimization

Para melhorar a qualidade, tente selecionar parâmetros de regularização usando pesquisa aleatória e validação cruzada de 3 vezes.

Esta execução necessita de bastante tempo de CPU e RAM (estamos ajustando um modelo 50 * 3 = 150 vezes), então pegue um chá e seja paciente, ou reduza n_iter em RandomizedSearchCV, ou ajuste o modelo apenas em um subconjunto de dados de treinamento.



Bibliotecas:
  scikit-Learn==0.23.2
  python 3.7


In [3]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

NameError: name 'labels' is not defined

### Vamos ver o melhor resultado:

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

### Verifique o espaço do parâmetro

Para isso plot um gráfico que mostra quais valores c1 e c2 têm RandomizedSearchCV. 
A cor vermelha significa melhores resultados, azul significa pior.

In [ ]:
_x = [s['c1'] for s in rs.cv_results_['params']]
_y = [s['c2'] for s in rs.cv_results_['params']]
_c = [s for s in rs.cv_results_['mean_test_score']]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

### Verifica o melhor estimador em nossos dados de teste

Como podemos ver, a qualidade é melhorada.

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))


### Vamos verificar o que o classificador aprendeu

In [ ]:
#Let’s check what classifier learned
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

### Verifique as características de cada token:
Podemos ver que, por exemplo, é muito provável que o início do nome de uma organização (B-ORG) seja seguido por um token dentro do nome da organização (I-ORG), mas transite para I-ORG de tokens com outros rótulos são penalizados.





In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])